In [ ]:
!pip install tensorflow==1.15.0

In [1]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [2]:
! python utils/generate_cifar10_tfrecords.py --data-dir=/tmp/cifar-10-data

Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
FloatProgress(value=0.0)
>> Downloading cifar-10-python.tar.gz 
Successfully downloaded cifar-10-python.tar.gz 170498071 bytes.
Generating /tmp/cifar-10-data/train.tfrecords


Generating /tmp/cifar-10-data/validation.tfrecords
Generating /tmp/cifar-10-data/eval.tfrecords
Removing original files.
Done!


In [3]:
! ls /tmp/cifar-10-data


eval.tfrecords	train.tfrecords  validation.tfrecords


In [6]:
%%sh

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
image=tf-cifar10-restnet

cd train

if [ "$image" == "" ]
then
    echo "Usage: $0 <image-name>"
    exit 1
fi

chmod +x cifar10/train
#chmod +x cifar10/serve

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"

# If the repository doesn't exist in ECR, create it.


aws ecr describe-repositories --repository-names "${image}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${image} .
docker tag ${image} ${fullname}

echo "tag finished...."

docker push ${fullname}

echo "push finished...."


Login Succeeded

Step 1/6 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:1.14.0-cpu-py36-ubuntu16.04
 ---> e6a210ff54e4
Step 2/6 : RUN apt-get update && apt-get install -y --no-install-recommends nginx curl
 ---> Using cache
 ---> 753b7238db78
Step 3/6 : RUN pip install gevent gunicorn
 ---> Using cache
 ---> 580a91923a0e
Step 4/6 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> e53705fb9f9c
Step 5/6 : COPY cifar10 /opt/ml/code
 ---> Using cache
 ---> 3d9275e40795
Step 6/6 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 8e902332e1d4
Successfully built 8e902332e1d4
Successfully tagged tf-cifar10-restnet:latest
tag finished....
The push refers to repository [731833107751.dkr.ecr.us-east-1.amazonaws.com/tf-cifar10-restnet]
5434f39cf1a7: Preparing
827f3a4d5aea: Preparing
3885d700ec51: Preparing
cc978a7bbd2a: Preparing
3a97a8d562fb: Preparing
cb460459ddc8: Preparing
b4064660a4cf: Preparing
b6e9883adafa: Preparing
9ee6d909e5a7: Preparing
e722e212cbab: Prepari

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

denied: User: arn:aws:sts::731833107751:assumed-role/AmazonSageMaker-ExecutionRole-20200129T155117/SageMaker is not authorized to perform: ecr:InitiateLayerUpload on resource: arn:aws:ecr:us-east-1:731833107751:repository/tf-cifar10-restnet
